In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nb

In [ ]:
server_path = "/Volumes/Scraplab/NNDb_data/ds002837/derivatives/"
derivative_path = "/Volumes/Scraplab/lindseytepfer/nndb/derivatives/"

sublist = [x for x in os.listdir(derivative_path) if ('sub-') in x] #grab all the subject IDs for easy filtering

## <font color='deeppink'>Trim .nii Files</font>
The NNDb data consists of 86 participants who watched 1 of 10 full-length films. In this study, we investigate 10 clips per movie, ranging from 15-25 seconds long. To analyze the brain data that corresponds with these clips, we trim the original .nii file into 10 shorter sub-files, according to when the participants would have watched each clip. Moreover, we shift the .nii trim timestamps by 4 seconds, to account for the hemodynamic response function in the brain. 

In [ ]:
# Get the name of all 100 clips into a list

clip_path = "/Users/f004p74/Documents/dartmouth/projects/NNDb/movie_clips/"
clip_dir = os.listdir(clip_path)
movie_list = [x for x in clip_dir if '.' not in x] #generates a list of all 10 movies
clip_list = []

zip_name, zip_start, zip_stop = [],[],[]

for movie in movie_list:
    clips = [x for x in os.listdir(clip_path+movie) if 'mp4' in x]
    
    for clip in clips:
        name = clip.split('.')[0]
        start = name.split('_')[1]
        stop = name.split('_')[2]
        clip_list.append(name)

        zip_name.append(name)
        zip_start.append(start)
        zip_stop.append(stop)


In [ ]:
movie_clip_dict = dict(zip(zip_name, zip(zip_start, zip_stop)))

In [ ]:
# Get a list of all the participant .nii files

server_path = "/Volumes/Scraplab/NNDb_data/ds002837/derivatives/"

niilist = []

for sub in sublist:
    img = [x for x in os.listdir(server_path+sub+'/func/') if '_bold_blur_censor_ica.nii.gz' in x]
    for x in img:
        niilist.append(x)

In [ ]:
for movie in movie_list:
    print("generating clips for: ", movie)

    movie_clip_data = [x for x in clip_list if movie in x] #[split_5609_5633 ... split_2416_2437]
    movie_neural_data = [x for x in niilist if movie in x] #[sub-71_task-split_bold_blur_censor_ica.nii.gz ...]

    for sub in movie_neural_data:
        sub_id = sub.split('_')[0]
        sub_img = nb.load(server_path+sub_id+os.sep+'func'+os.sep+sub) #../derivatives/sub-71/func/sub-71_task-split_bold_blur_censor_ica.nii.gz

        for clip in movie_clip_data: #each clip takes apprx 2m
            start = int(clip_dict[clip][0]) + 4
            stop = int(clip_dict[clip][1]) + 4
            fname = sub_id+"_"+movie+"_"+str(start)+"_"+str(stop) #sub-71_split_5613_5637
            print(fname)
            
            clip_slice = sub_img.slicer[:,:,:,start:stop]

            nb.save(clip_slice, derivative_path+sub_id+os.sep+fname+".nii.gz")

## <font color='deeppink'>Mask schaefer atlas to trimmed nii files</font>

In [ ]:
import nibabel as nb
import nilearn
from nilearn import datasets
import nilearn.image as image
from nilearn.maskers import NiftiMasker

In [ ]:
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=1,
                                                    data_dir=None, base_url=None, resume=True, verbose=1)
'''
From the documentation:
The list of labels does not contain ‘Background’ by default. 
To have proper indexing, you should either manually add ‘Background’ to the list of labels:
'''

schaefer_atlas.labels = np.insert(schaefer_atlas.labels, 0, "Background")

In [ ]:
# holding all 400 parcel masks in memory; takes apprx 2m13s
mask_list = []

for p in range(1,402): #402

    try:
        parcel = nilearn.image.new_img_like(schaefer_atlas.maps, nilearn.image.get_data(schaefer_atlas.maps) == p) #hold the parcel masks in memory 
        masker = NiftiMasker() 

        parcel_mask = masker.fit(parcel)
        mask_list.append(parcel_mask)
    
    except: 
        print("out of range, p=", p)
        continue

In [ ]:
for s in sub_list:
    print("Starting with subject: ", s)
    sub_clips = os.listdir(derivative_path+s)
    sub_clips = [x for x in sub_clips if 'sub' in x]
    sub_clips.sort() # super important, it has to be in ascending order. 

    for c in sub_clips[1:]: #2m 13s per clip
        clip_slice = nb.load(derivative_path+s+os.sep+c)
        clip_avg = image.mean_img(clip_slice)

        clip_name = c.replace('.', '_')
        clip_name = clip_name.split('_')[1:4]
        clip_name = "_".join(clip_name)

        data_list = []

        for ix,mask in enumerate(mask_list):
            try:
                roi_data = mask.transform_single_imgs(clip_avg)
                data_list.append(roi_data[0])
            except:
                print("index: ", ix)
                continue

        df = pd.DataFrame(data_list)
        df.to_csv(derivative_path+s+os.sep+"clip_voxels/"+str(clip_name)+".csv", index=False)


# <font color='deeppink'>Creating Similartiy Matrices</font>
<font color='deeppink'>Create a 10x10 similarity matrix: </font> for each parcel, I want to create a similarity matrix among all the 10 clips the participant watched.

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
for sub in sublist:
    os.mkdir('/Volumes/Scraplab/lindseytepfer/nndb/derivatives/'+sub+'/parcel_correlations')

In [ ]:
#takes too long to run locally
for s in sublist:
    spath = derivative_path+s+'/masked_nii/'
    correlation_files = [x for x in  os.listdir(spath) if '.csv' in x]

    df_list = [pd.read_csv(spath+x) for x in correlation_files]

    for i in range(len(df_list[0])): 
        mat = pd.DataFrame()
        
        for ix, df in enumerate(df_list): # go into each df 
            voxels = df.iloc[i]
            mat[correlation_files[int(ix)]] = voxels

        mat.dropna(inplace=True)

        #produces a 10x10 correlation matrix as a .csv for each parcel's voxel signal across the 10 movie clips
        distance_matrix = pd.DataFrame(pairwise_distances(mat.T, metric='correlation'))
        distance_matrix.to_csv(derivative_path+s+'/parcel_correlations/'+str(i)+"_correlation.csv", index=False, header=False)


<font color='deeppink'>Import the 100x100 behavioral distance matrix.</font>

In [ ]:
import nilearn
import statsmodels.api as sm
from nilearn.connectome import ConnectivityMeasure

In [ ]:
iv_list = []

for x in os.listdir(derivative_path+"behavioral_matrices"):
    if '.csv' in x:
        print(x)
        df = pd.read_csv(derivative_path+"behavioral_matrices/"+x)
        iv_list.append(df)
    else:
        continue

When we check whether there is co-linearity between the IV's, we find very high values:

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

In [ ]:
df = pd.read_csv("movie_tcv_ratings.csv")
df.head()

In [ ]:
X = df[['watched', 'social', 'feeling', 'tension', 'conflict', 'violence', 'count_people', 'count_interactions']]
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range (len(X.columns))]

In [ ]:
vif_data

### <font color='hotpink'> Run simple regressions.</font> 
 Next, we run simple regressions to get zero-order estimates for each of our independent variables (e.g., conflict, violence, tension). 

In [ ]:
# takes X minutes to run locally
for s in sublist:
    spath = derivative_path+s # ~/Volumes/Scraplab/S/lindseytepfer/nndb/derivatives/sub-s
    smovielist = [x for x in os.listdir(spath) if 'sub' in x] #[sub-s_500daysofsummer_4841_4868.nii.gz, ....]
    smovie = smovielist[0].split('_')[1] # 500daysofsummer

    parcel_list = [x for x in  os.listdir(spath+'/parcel_correlations/') if 'correlation' in x] # 400 n_correlation.csv files 
    parcel_names = [x.split('_')[0] for x in parcel_list] # 0-399

    results_df = pd.DataFrame(columns=[['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']])

    for p in parcel_list: #e.g., ['168_correlation.csv']
        parcel_df = pd.read_csv(spath+'/parcel_correlations/'+p, header=None)
        y = nilearn.connectome.sym_matrix_to_vec(np.array(parcel_df), discard_diagonal=True) # lower triangle, shape (45,)

        all_iv = np.empty((45,8))
        
        for i in range(len(iv_list)): #iv_list = [conflict_distance_matrix.csv, violence_distance_matrix.csv ...]
            df = iv_list[i]
            filtered_df = df[(df.movie == smovie)]
            col = [str(x) for x in list(filtered_df.index)]
            
            x_100 = filtered_df.drop(['movieID','movie'], axis=1)
            x = x_100[col] #ensures corresponding movie columns
            x_lower = nilearn.connectome.sym_matrix_to_vec(np.array(x), discard_diagonal=True) # lower triangle

            all_iv[:,i] = x_lower
        
        iv_corrcoefs = []

        for iv in range(8):
            single_iv = all_iv[:,iv]
            r = np.corrcoef(single_iv, y)[0][1]
            iv_corrcoefs.append(r)

        results_df.loc[len(results_df)] = iv_corrcoefs
    
    results_df['subject'] = s
    results_df['movie'] = smovie
    results_df['parcel'] = parcel_names

    results_df.to_csv(spath+'/results.csv')

In [ ]:
os.listdir(derivative_path+s+"")

### <font color='deeppink'> Calculating Cohen's d</font>

In [ ]:
#first, net each participant's model estimate 
# model_estimates = []

# for s in sublist:
#     model_estimates.append(pd.read_csv(derivative_path+s+'/results.csv'))

# combined_estimates = pd.concat(model_estimates)
# combined_estimates.reset_index(inplace=True, drop=True)
# combined_estimates.drop(['Unnamed: 0'],inplace=True, axis=1)
# combined_estimates.to_csv(derivative_path+"/combined_estimates.csv")

In [ ]:
combined_estimates = pd.read_csv(derivative_path+'/combined_estimates.csv')

In [ ]:
out = pd.DataFrame(columns=[['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']])

for movie in movie_list[0:1]:
    movie_estimates = combined_estimates[(combined_estimates.movie == movie)]

    for i in range(400):
        parcel = movie_estimates.loc[movie_estimates.parcel == i ] #get the regression model estimates for a single parcel, within a movie

        cols = ['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']
        dvals = []

        for col in cols:
            dvals.append(parcel[col].mean()/ parcel[col].std())

        out.loc[len(out)] = dvals

out.to_csv(derivatives+'/parcel_cohensd.csv', index=False)





In [ ]:
# convert betas to cohen's d value -- done

out = pd.DataFrame(columns=[['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']])

for i in range(400):
    parcel = combined_estimates.loc[combined_estimates.parcel == i]

    cols = ['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']
    dvals = []
    
    for col in cols:
        dvals.append(parcel[col].mean()/ parcel[col].std())

    out.loc[len(out)] = dvals

out.to_csv(derivatives+'/parcel_cohensd.csv', index=False)

### <font color='deeppink'> Permutation Analyses</font>

Next, we will bootstrap the participants and permute the cohen's d values.

In [ ]:
import os
import pandas as pd
import numpy as np

derivatives = "/Users/f004p74/Documents/dartmouth/projects/NNDb/derivatives/"

sub_list = os.listdir(derivatives)
sub_list = [x for x in sub_list if 'sub' in x] #remove hidden files

results = pd.read_csv(derivatives+'/complete_betas.csv')

In [ ]:
perm_mat = np.empty((400,10000,8))

In [ ]:
columns = ['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']

for i in range(400): #for each parcel...

    np.random.seed(0)

    parcel = results.loc[results.parcel == i]

    #bootstrap and permute the parcel dataframe
    for p in range(10000):

        sample_movies = parcel.sample(n=10, replace=True) #returns ... ?

        participant_n = sample_movies['subject'].unique() # a list of all the participants

        #set up the sign flipping at the participant level:
        signs_arr = np.random.choice((-1,1), len(participant_n), replace=True) # returns an array of 1's and -1's of length 86
        
        for ix, subject in enumerate(participant_n): #create the sign-flipped dataframe

            filtered = sample_movies[(sample_movies.subject == subject)]
            filtered.loc[:, columns] = filtered[columns].apply(lambda x: x * signs_arr[ix], axis=1)
            sample_movies[(sample_movies.movie == subject)] = filtered

        #now with the bootstrap and the permutation orders set up, calculate new cohen d values:
        perm_mat[i,p] = np.array(np.mean(sample_movies[columns],axis=0)/np.std(sample_movies[columns],axis=0))

np.save(derivatives+'/permutation_matrix', perm_mat)


In [ ]:
columns = ['conflict', 'watched', 'npeople', 'violence', 'ninteractions', 'feeling', 'tension', 'issocial']

for i in range(400): #for each parcel...

    np.random.seed(0)

    parcel = results.loc[results.parcel == i]

    #bootstrap and permute the parcel dataframe
    for p in range(10000):

        boot_sample = parcel.sample(n=86, replace=True)
        boot_movies = boot_sample['movie'].unique() # an array of movie names

        #set up the sign flipping for each parcel sample's set of movies:
        signs_arr = np.random.choice((-1,1), len(boot_movies), replace=True) # returns an array of 1's and -1's
        
        for ix, movie in enumerate(boot_movies): #create the sign-flipped dataframe

            filtered = boot_sample[(boot_sample.movie == movie)]
            filtered.loc[:, columns] = filtered[columns].apply(lambda x: x * signs_arr[ix], axis=1)
            boot_sample[(boot_sample.movie == movie)] = filtered

        #now with the bootstrap and the permutation orders set up, calculate new cohen d values:
        perm_mat[i,p] = np.array(np.mean(boot_sample[columns],axis=0)/np.std(boot_sample[columns],axis=0))

np.save(derivatives+'/permutation_matrix', perm_mat)
